### Trabalho Final
***
- 1606881 | Bruna Mirele Martins da Silveira
- 1606894 | Caio Wey Barros
- 1606966 | Gustavo Matheus Santos Penedo
***

#### Objetivos de Ensino

#### Exercitar os seguintes conceitos:
1. Coleta de dados
2. Analisar e explorar os conjuntos de dados
3. Analisar e realizar tratamento de dados se necessário
4. Realizar agrupamentos de dados
5. Analisar resultados obtidos
6. Responder as questões objetivas

#### Enunciado da Atividade Prática: Análise de Dados

Você foi contratado para criar uma automação de fluxo de dados completa para dados clínicos dos pacientes em um projeto de pesquisa de saúde. Seus dados incluem três conjuntos principais: dados_clinicos, dados_pacientes e dados_estados. Cada conjunto de dados possui as seguintes colunas:
1. **dados_clinicos**: contém informações sobre os pacientes como id_cliente, peso, colesterol e ganero.
2. **dados_pacientes**: fornece detalhes sobre os pacientes como id_cliente, idade, classe_trabalho, escolaridade, id_estado, estado_civil, raca, qtde_filhos e salario.
3. **dados_estados**: oferece informações sobre os estados, incluindo id_estado, sigla, estado, regiao e pais.

Os datasets utilizados no trabalho podem ser obtidos nesse [link](https://leandrolessa.com.br/wp-content/uploads/2024/06/dados_clinicos_pacientes.zip).

#### Tarefas:

In [6]:
# Bibliotecas
import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
load_dotenv('../config/.env')

True

**1. Coleta de Dados**

Realize a coleta de dados dos pacientes disponibilizados.

In [145]:
# Coleta de dados clinicos
df_clinicos = pd.read_csv('../datasets/dados_clinicos/dados_clinicos.csv', sep='|', encoding='latin1')

# Coleta de dados estado região
df_estado_regiao = pd.read_csv('../datasets/dados_estado_regiao/estado_regiao.csv', sep=';', encoding='latin1')

# Coleta de dados pacientes
df_pacientes = pd.read_csv('../datasets/dados_pacientes/dados_pacientes.csv', sep=';', encoding='utf-8')

**2. Integração dos Dados**

Realize um inner join entre dados_clinicos, dados_pacientes e dados_estados para obter uma visão completa dos dados dos pacientes.

In [146]:
# Integração de dados
df = pd.merge(
    left = df_pacientes,
    right = df_clinicos,
    on = 'id_cliente',
    how = 'inner'
)

df = pd.merge(
    left = df,
    right = df_estado_regiao,
    on = 'id_estado',
    how = 'inner'
)

df.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario,peso,colesterol,genero,sigla,estado,regiao,pais
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754,152.7,203.9,Masculino,AL,Alagoas,Nordeste,Brasil
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923,184.8,233.8,Masculino,SC,Santa Catarina,Sul,Brasil
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100,152.1,210.4,Masculino,AM,Amazonas,Norte,Brasil
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100,139.8,196.8,Masculino,SC,Santa Catarina,Sul,Brasil
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430,182.4,215.8,Feminino,PB,Paraíba,Nordeste,Brasil


**3. Correção dos Dados**

Identifique e corrija dados inconsistentes, como valores ausentes, duplicados ou incorretos.

In [147]:
# Remoção de duplicados
df = df.drop_duplicates()

# Cálculo de moda e mediana
mediana_peso = df['peso'].median().round(2)
moda_genero = df['genero'].mode()
moda_classe_trabalho = df['classe_trabalho'].mode()
mediana_qtde_filhos = int(df['qtde_filhos'].median())

# Correção de valores ausentes
df['peso'] = df['peso'].fillna(value=mediana_peso)
df['genero'] = df['genero'].fillna(value=moda_genero[0])
df['classe_trabalho'] = df['classe_trabalho'].fillna(value=moda_classe_trabalho[0])
df['qtde_filhos'] = df['qtde_filhos'].fillna(value=mediana_qtde_filhos)

# Correção de valores incorretos
df['peso'] = df['peso'].round(2)
df['colesterol'] = df['colesterol'].round(2)
df['qtde_filhos'] = df['qtde_filhos'].astype(int)

df.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario,peso,colesterol,genero,sigla,estado,regiao,pais
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2,4754,152.7,203.9,Masculino,AL,Alagoas,Nordeste,Brasil
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1,3923,184.8,233.8,Masculino,SC,Santa Catarina,Sul,Brasil
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0,1100,152.1,210.4,Masculino,AM,Amazonas,Norte,Brasil
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1,1100,139.8,196.8,Masculino,SC,Santa Catarina,Sul,Brasil
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0,3430,182.4,215.8,Feminino,PB,Paraíba,Nordeste,Brasil


**4. Armazenamento de Dados**

Desenvolva o diagrama de entidade-relacionamento entre as tabelas e persista os dados no banco de dados MySQL.

In [8]:
# Conexão com o banco de dados
def conectar_banco_mysql():
    try:
        user = os.environ['user_db']
        password = os.environ['password_db']
        host = os.environ['host']
        conexao = f'mysql://{user}:{password}@{host}'
        engine = create_engine(conexao)
        conn = engine.connect()
        print('Conexão realizada com sucesso!')
        return conn
    except Exception as e:
        print(f'Não foi possível fazer conexão com o banco de dados. ERRO: {e}')

# Validação da conexão com o servidor de banco de dados
conn = conectar_banco_mysql()

# Criação do banco de dados
try:
    database = 'pacientes'
    query = f'create schema if not exists {database}'
    conn.execute(text(query))
    print('Banco de dados criado com sucesso!')
    conn.commit()
except Exception as e:
    print(f'Não foi possível criar o banco de dados. ERRO: {e}')

# Execução do script de criação das tabelas e seus relacionamentos
with open('../script_DB/create_database_paciente.sql', 'r') as file:
    query = file.read()

    try:
        conn.execute(text(query))
        print('Tabelas criadas com sucesso!')
        conn.commit()
    except Exception as e:
        print(f'Não foi possível criar as tabelas. ERRO: {e}')

# Inserção de registros no banco de dados
def inserir_dados_tabelas(lista_dados, nome_tabela):
    coluna_tabela = nome_tabela[3:]

    for registro in lista_dados:
        try:
            query = f'''
                INSERT INTO {nome_tabela} ({coluna_tabela})
                VALUES ("{registro}")
            '''
            conn.execute(text(query))
            conn.commit()
            print(f'Registro inserido com sucesso: {registro}')
        except Exception as e:
            print(f'Não foi possível inserir o registro: {registro}. ERRO: {e}')
            conn.rollback()

Conexão realizada com sucesso!
Banco de dados criado com sucesso!
Tabelas criadas com sucesso!


**5. Consultas SQL**

Após o armazenamento dos dados, responda às perguntas do questionário utilizando SQL.

In [ ]:
# Listas
lista_classe_trabalho = df['classe_trabalho'].unique()
lista_escolaridade = df['escolaridade'].unique()
lista_raca = df['raca'].unique()
lista_estado_civil = df['estado_civil'].unique()

lista_insert = [
    ('tb_classe_trabalho', lista_classe_trabalho),
    ('tb_escolaridade', lista_escolaridade),
    ('tb_raca', lista_raca),
    ('tb_estado_civil', lista_estado_civil)
]

# Inserção dos dados nas tabelas
for tabela, dados in lista_insert:
    inserir_dados_tabelas(
        lista_dados = dados,
        nome_tabela = tabela
    )

# Dados estados
colunas = df_estado_regiao.columns.to_list()

dados_estados = df[colunas].drop_duplicates()

# Inserção dos dados estados na tabela
for registro in dados_estados.itertuples():
    cod_estado = registro.id_estado
    estado = registro.estado
    sigla = registro.sigla
    regiao = registro.regiao
    pais = registro.pais

    try:
        query = f'''
            INSERT INTO TB_ESTADO (cod_estado, estado, sigla, regiao, pais)
            VALUES ({cod_estado}, "{estado}", "{sigla}", "{regiao}", "{pais}")
        '''
        conn.execute(text(query))
        conn.commit()
    except Exception as e:
        print(f"Não foi possível inserir o registro {registro}. ERRO: {e}")
        conn.rollback()

# Criação da tabela stage
try:
    df.to_sql(
        name="stg_pacientes",
        con=conn,
        schema="pacientes",
        index=False
    )
except Exception as e:
    print(f"Não foi possível inserir os dados na tabela. ERRO: {e}")

# Inserção dos dados paciente na tabela
query = f"""
INSERT INTO tb_paciente (
    cod_paciente,
    cod_estado_civil,
    cod_raca,
    cod_escolaridade,
    cod_classe_trabalho,
    cod_estado,
    genero,
    idade,
    qtd_filho,
    salario,
    peso,
    colesterol
)

(
SELECT pac.id_cliente,
    eci.cod_estado_civil,
    rac.cod_raca,
    esc.cod_escolaridade,
    ctr.cod_classe_trabalho,
    est.cod_estado,
    pac.genero,
    pac.idade,
    pac.qtde_filhos,
    pac.salario,
    pac.peso,
    pac.colesterol
FROM stg_pacientes AS pac
INNER JOIN tb_estado_civil AS eci ON eci.estado_civil = pac.estado_civil
INNER JOIN tb_raca AS rac ON rac.raca = pac.raca
INNER JOIN tb_escolaridade AS esc ON esc.escolaridade = pac.escolaridade
INNER JOIN tb_classe_trabalho AS ctr ON ctr.classe_trabalho = pac.classe_trabalho
INNER JOIN tb_estado AS est ON est.estado = pac.estado
)
"""
try:
    conn.execute(text(query))
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print(f"Não foi possível inserir os dados na tabela. ERRO: {e}")
    conn.rollback()

Registro inserido com sucesso: Servidor Público
Registro inserido com sucesso: Autônomo
Registro inserido com sucesso: Funcionário Setor Privado
Registro inserido com sucesso: Funcionário Público
Registro inserido com sucesso: Empresário
Registro inserido com sucesso: Estagiário
Registro inserido com sucesso: Desempregado
Registro inserido com sucesso: MEI
Registro inserido com sucesso: Aposentado
Registro inserido com sucesso: Menor Aprendiz
Registro inserido com sucesso: Ensino Médio Completo
Registro inserido com sucesso: Superior Incompleto
Registro inserido com sucesso: Ensino Médio Incompleto
Registro inserido com sucesso: Mestrado
Registro inserido com sucesso: Ensino Fundamental Completo
Registro inserido com sucesso: Superior Completo
Registro inserido com sucesso: Pós-Graduado
Registro inserido com sucesso: Ensino Fundamental Incompleto
Registro inserido com sucesso: Doutorado
Registro inserido com sucesso: Analfabeto
Registro inserido com sucesso: Branco
Registro inserido co

#### **ATENÇÃO PARA TRATAMENTO DE DADOS**

Avaliem se será necessário realizar tratamento de dados ausentes nos datasets disponibilizados.

**Instruções para correção de dados ausentes**

- Moda para Variáveis Categóricas: Preencher valores ausentes nas colunas categóricas com a moda.
- Mediana para Variáveis Numéricas: Preencher valores ausentes nas colunas numéricas com a mediana.

#### Utilize o diagrama de entidade e relacionamento

![Diagrama de entidade e relacionamento](../doc/Diagrama%20de%20Entidade%20e%20Relacionamento.png)

#### Dicas do professor:

1. Antes de finalizar suas respostas, certifique-se de verificar se o gabarito está corretamente preenchido.
2. Preste atenção aos requisitos específicos de cada questão para garantir uma resposta adequada.
3. Elimine dados duplicados.
4. Lembre-se de que os dados fornecidos nos datasets são fictícios e não refletem dados do mundo real.
5. Siga rigorosamente todas as etapas descritas no enunciado das questões para uma análise completa e precisa.

#### Perguntas do Trabalho

**1. Qual é a média e o desvio padrão do colesterol das pessoas que têm peso superior a 120kg e residem na região Sul do Brasil?**

In [ ]:
query = '''
SELECT AVG(pac.colesterol) AS media_colesterol,
	STDDEV_POP(pac.colesterol) AS desvio_padrao_colesterol
FROM tb_paciente as pac
INNER JOIN tb_estado AS est ON est.cod_estado = pac.cod_estado
WHERE pac.peso > 120
AND est.regiao = "Sul"
'''

try:
    result = pd.read_sql_query(query, conn)
    print(result)
except Exception as e:
    print('Não foi possível executar a query. ERRO: {e}')

   media_colesterol  desvio_padrao_colesterol
0        206.955385                 21.866546


In [ ]:
SELECT AVG(pac.colesterol) AS media_colesterol,
	STDDEV_POP(pac.colesterol) AS desvio_padrao_colesterol
FROM tb_paciente as pac
INNER JOIN tb_estado AS est ON est.cod_estado = pac.cod_estado
WHERE pac.peso > 120
AND est.regiao = "Sul"

**2. Qual é a classe de trabalho com maior número de pessoas e qual é o número correspondente na base de dados?**

In [ ]:
SELECT ctr.classe_trabalho,
    COUNT(1) AS numero_pessoas
FROM tb_paciente AS pac
INNER JOIN tb_classe_trabalho AS ctr ON ctr.cod_classe_trabalho = pac.cod_classe_trabalho
GROUP BY ctr.classe_trabalho
ORDER BY numero_pessoas DESC
LIMIT 1

**3. Qual é a classe de trabalho que apresenta a maior média de salários e qual é o valor associado para as pessoas do estado de Pernambuco?**

In [ ]:
-- Classe de trabalho com maior média de salários
SELECT AVG(pac.salario) AS media_salario,
	ctr.classe_trabalho
FROM tb_paciente AS pac
INNER JOIN tb_classe_trabalho AS ctr ON ctr.cod_classe_trabalho = pac.cod_classe_trabalho
GROUP BY ctr.classe_trabalho
LIMIT 1

-- Valor associado para as pessoas do estado de Pernambuco
SELECT AVG(pac.salario) AS media_salario,
    est.estado
FROM tb_paciente AS pac
INNER JOIN tb_estado AS est ON est.cod_estado = pac.cod_estado
WHERE est.estado = "Pernambuco"

**4. Qual estado civil possui a menor quantidade de filhos e qual é esse número?**

In [ ]:
SELECT SUM(pac.qtd_filho) AS quantidade_filhos,
	eci.estado_civil
FROM tb_paciente AS pac
INNER JOIN tb_estado_civil AS eci ON eci.cod_estado_civil = pac.cod_estado_civil
GROUP BY eci.estado_civil
ORDER BY quantidade_filhos
LIMIT 1

**5. Quantas pessoas casadas possuem filhos e têm um salário acima de 3000?**

In [ ]:
SELECT COUNT(1) AS numero_pessoas
FROM tb_paciente AS pac
INNER JOIN tb_estado_civil AS eci ON eci.cod_estado_civil = pac.cod_estado_civil
WHERE eci.estado_civil = "Casado"
AND pac.qtd_filho > 0
AND pac.salario > 3000

**6. Qual é a média salarial das pessoas casadas que possuem ensino superior completo e trabalham como Funcionário do Setor Privado?**

In [ ]:
SELECT AVG(pac.salario) AS media_salario
FROM tb_paciente AS pac
INNER JOIN tb_estado_civil AS eci ON eci.cod_estado_civil = pac.cod_estado_civil
INNER JOIN tb_escolaridade AS esc ON esc.cod_escolaridade = pac.cod_escolaridade
INNER JOIN tb_classe_trabalho AS ctr ON ctr.cod_classe_trabalho = pac.cod_classe_trabalho
WHERE eci.estado_civil = "Casado"
AND esc.escolaridade = "Superior Completo"
AND ctr.classe_trabalho = "Funcionário Setor Privado"

**7. Qual estado civil possui a menor quantidade de filhos e qual é esse número?**

In [ ]:
-- REPETIDA

**8. Quantas pessoas casadas possuem filhos e têm um salário acima de 3000?**

In [ ]:
SELECT COUNT(1)
FROM tb_paciente AS pac
INNER JOIN tb_estado_civil AS eci ON eci.cod_estado_civil = pac.cod_estado_civil
WHERE eci.estado_civil = "Casado"
AND pac.qtd_filho > 0
AND pac.salario > 3000

**9. Qual é a soma das idades de todas as mulheres solteiras que residem em Santa Catarina?**

In [ ]:
SELECT pac.genero,
	eci.estado_civil,
    est.estado,
    SUM(pac.idade)
FROM tb_paciente AS pac
INNER JOIN tb_estado_civil AS eci ON eci.cod_estado_civil = pac.cod_estado_civil
INNER JOIN tb_estado AS est ON est.cod_estado = pac.cod_estado
WHERE pac.genero = "Feminino"
AND eci.estado_civil = "Solteiro"
AND est.estado = "Santa Catarina"

**10. Qual o estado que possui a maior média de peso?**

In [ ]:
SELECT est.estado,
	AVG(pac.peso) AS media_peso
FROM tb_paciente AS pac
INNER JOIN tb_estado AS est ON est.cod_estado = pac.cod_estado
GROUP BY est.estado
ORDER BY media_peso DESC
LIMIT 1